In [ ]:
import pandas as pd


clients = pd.read_csv("datos/caso_2/clients.csv")
depots = pd.read_csv("datos/caso_2/depots.csv")
depots = depots.drop(columns=["DepotID"])
vehicles = pd.read_csv("datos/caso_2/vehicles.csv")

# clients = pd.read_csv("datos/caso_2_petite/clients.csv")
# depots = pd.read_csv("datos/caso_2_petite/depots.csv")
# depots = depots.drop(columns=["DepotID"])
# vehicles = pd.read_csv("datos/caso_2_petite/vehicles.csv")



print(clients.head())
print(depots.head())
print(vehicles.head())


   LocationID  ClientID   Latitude  Longitude  Demand   TimeWindow
0           2         1  12.341057 -71.723837    25.0  13:20-13:50
1           3         2  12.310993 -71.719252     9.0  13:30-14:00
2           4         1  12.341057 -71.723837    25.0  13:20-13:50
3           5         1  12.341057 -71.723837    25.0  13:20-13:50
4           6         2  12.310993 -71.719252     9.0  13:30-14:00
   LocationID  Latitude  Longitude
0           1  12.31846 -71.716203
   VehicleID   Type  Capacity  Range  Speed
0          1    4x4     200.0  804.0    NaN
1          2  drone      25.0  480.0  135.0
2          3    4x4       1.0  804.0    NaN
3          4  drone       0.0  480.0  135.0


In [320]:
from haversine import haversine


def dehhmm_a_minutos(hhmm):
    """
    Convierte un tiempo en formato de horas y minutos (HHMM) a minutos.
    :param dehhmm: Tiempo en formato HHMM.
    :return: Tiempo en minutos.
    """
    horas = hhmm // 100
    minutos = hhmm % 100
    return horas * 60 + minutos


# recibe "hh:mm-hh:mm" y devuelve una lista de tuplas con los tiempos en minutos
def convertir_hora_a_minutos(rango_horas):
    """
    Convierte un rango de horas en formato "hh:mm-hh:mm" a una lista de tuplas con los tiempos en minutos.
    :param rango_horas: Rango de horas en formato "hh:mm-hh:mm".
    :return: tiempos en minutos.
    """
    inicio, fin = rango_horas.split('-')
    inicio_min = dehhmm_a_minutos(int(inicio.replace(':', '')))
    fin_min = dehhmm_a_minutos(int(fin.replace(':', '')))
    return [inicio_min, fin_min]

# haversine

def haversine_distance(coord1, coord2):
    """
    Calculate the haversine distance between two coordinates.
    :param coord1: Tuple of (latitude, longitude) for the first coordinate.
    :param coord2: Tuple of (latitude, longitude) for the second coordinate.
    :return: Distance in kilometers.
    """
    return haversine(coord1, coord2)

In [321]:

depots = depots.rename(columns={"LocationID": "id"})
clients = clients.rename(columns={"LocationID": "id"})
depots["id"] = 0 
depots["Demand"] = 0

clients = clients.reset_index(drop=True)
clients["id"] = clients.index + 1  

depots = depots[["id", "Longitude", "Latitude", "Demand"]]
clients = clients[["id", "Longitude", "Latitude", "Demand"]]

nodes = pd.concat([depots, clients], ignore_index=True)


N = nodes["id"].tolist()

C = clients["id"].tolist()

De = nodes.set_index("id")["Demand"].to_dict()

depot = 0

V = vehicles["VehicleID"].tolist()
P = vehicles.set_index("VehicleID")["Capacity"].to_dict()
M = vehicles.set_index("VehicleID")["Range"].to_dict()

tipo_vehiculo = [ 1 if v == "4x4" else 2 for v in vehicles["Type"].tolist() ] # 1: carros, 2: drones

print("Tipo de vehiculo:", tipo_vehiculo)

# Adaptaciones caso 2

K = [1,2] # 1: carros, 2 drones

distances = {}

for i in range(len(nodes)):
    for j in range(len(nodes)):
        for k in K:
            if i != j and k == 1: # solo para carros
                coord1 = (nodes.iloc[i]["Latitude"], nodes.iloc[i]["Longitude"])
                coord2 = (nodes.iloc[j]["Latitude"], nodes.iloc[j]["Longitude"])
                distances[(nodes.iloc[i]["id"], nodes.iloc[j]["id"], k)] = haversine_distance(coord1, coord2) *1.3 # 30% más por ser un carro
            if i != j and k == 2: # solo para drones
                coord1 = (nodes.iloc[i]["Latitude"], nodes.iloc[i]["Longitude"])
                coord2 = (nodes.iloc[j]["Latitude"], nodes.iloc[j]["Longitude"])
                distances[(nodes.iloc[i]["id"], nodes.iloc[j]["id"], k)] = haversine_distance(coord1, coord2) # No hay penalización de distancia



print("Nodos:", N)
print("Clientes:", C)
print("Vehículos:", V)
print("Capacidades:", P)
print("Alcances:", M)
print("Demanda:", De)



Tipo de vehiculo: [1, 2, 1, 2]
Nodos: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Clientes: [1, 2, 3, 4, 5, 6, 7, 8, 9]
Vehículos: [1, 2, 3, 4]
Capacidades: {1: 200.0, 2: 25.0, 3: 1.0, 4: 0.0}
Alcances: {1: 804.0, 2: 480.0, 3: 804.0, 4: 480.0}
Demanda: {0: 0.0, 1: 25.0, 2: 9.0, 3: 25.0, 4: 25.0, 5: 9.0, 6: 25.0, 7: 25.0, 8: 9.0, 9: 25.0}


In [322]:
print(distances)

{(np.float64(0.0), np.float64(1.0), 1): 3.439820297737471, (np.float64(0.0), np.float64(1.0), 2): 2.6460156136442086, (np.float64(0.0), np.float64(2.0), 1): 1.1621579203687056, (np.float64(0.0), np.float64(2.0), 2): 0.8939676310528504, (np.float64(0.0), np.float64(3.0), 1): 3.439820297737471, (np.float64(0.0), np.float64(3.0), 2): 2.6460156136442086, (np.float64(0.0), np.float64(4.0), 1): 3.439820297737471, (np.float64(0.0), np.float64(4.0), 2): 2.6460156136442086, (np.float64(0.0), np.float64(5.0), 1): 1.1621579203687056, (np.float64(0.0), np.float64(5.0), 2): 0.8939676310528504, (np.float64(0.0), np.float64(6.0), 1): 3.439820297737471, (np.float64(0.0), np.float64(6.0), 2): 2.6460156136442086, (np.float64(0.0), np.float64(7.0), 1): 3.439820297737471, (np.float64(0.0), np.float64(7.0), 2): 2.6460156136442086, (np.float64(0.0), np.float64(8.0), 1): 1.1621579203687056, (np.float64(0.0), np.float64(8.0), 2): 0.8939676310528504, (np.float64(0.0), np.float64(9.0), 1): 3.439820297737471, (n

# MODELO

In [323]:
from pyomo.environ import *

model = ConcreteModel()

costo_por_km = 0.57
cost_dict = {(i, j, k): distances[i, j,k] * costo_por_km for i in N for j in N for k in K if i != j}

model.N = Set(initialize=N)       # Nodos 
model.C = Set(initialize=C)        # Clientes
model.V = Set(initialize=V)        # Vehiculos
model.K = Set(initialize=K)        # Tipos de vehiculos (carros y drones)
model.depot = Param(initialize=depot) # Deposito

model.d = Param(model.N, model.N,model.K, initialize=distances, within=NonNegativeReals) # Distancia
model.De = Param(model.N, initialize=De, within=NonNegativeReals, default=0)      # Demanda cliente
model.P = Param(model.V, initialize=P, within=NonNegativeReals)      # Capacidad vehiculo              
model.M = Param(model.V, initialize=M, within=NonNegativeReals) # autonomia vehiculo	
model.cost = Param(model.N, model.N,model.K, initialize=cost_dict, within=NonNegativeReals, default=0) # Costo por arco



model.x = Var(model.N, model.N, model.V, domain=Binary) # Variable binaria que indica si el vehiculo k viaja del nodo i al nodo j
model.c = Var(model.N, model.N, model.V, domain=NonNegativeReals) # Variable continua que indica el costo del arco (i,j) para el vehiculo k
model.u = Var(model.N, model.V, within=NonNegativeIntegers, bounds=(1, len(N))) # subtours
model.cv = Var(model.V, domain=NonNegativeReals) # carga que lleva el vehiculo k
model.ve = Var(model.V, domain=Binary) # variable binaria que indica si el vehiculo k es utilizado o no


# Objective function
# Minimize the total cost of the routes taken by all vehicles
model.obj = Objective(
    expr=sum(model.cost[i, j, tipo_vehiculo[k-1] ] * model.x[i, j, k] for k in model.V for i in model.N for j in model.N if i != j),
    sense=minimize
)


# Constraints

# Visit once constraint: each node must be visited exactly once by one vehicle
model.visit_once = ConstraintList()
for i in model.C:
    model.visit_once.add(sum(model.x[i, j, k] for j in model.N for k in model.V if i != j) == 1)

# Flujos

model.flows = ConstraintList()
for k in model.V:
    model.flows.add(sum(model.x[0, j, k] for j in model.N if j != 0) == model.ve[k])  
    model.flows.add(sum(model.x[j, 0, k] for j in model.N if j != 0) == model.ve[k])  

model.start_node = ConstraintList()
for k in model.V:
    model.start_node.add(sum(model.x[0, j, k] for j in model.N if j != 0) == model.ve[k])
    
model.end_node = ConstraintList()
for k in model.V:
    model.end_node.add(sum(model.x[j, 0, k] for j in model.N if j != 0) == model.ve[k])
    
model.flow = ConstraintList()
for k in model.V:
    for i in model.C:
        model.flow.add(sum(model.x[i, j, k] for j in model.N if i != j) ==
                       sum(model.x[j, i, k] for j in model.N if i != j))

# Subtour elimination constraints
model.mtz = ConstraintList()
for k in model.V:
    for i in model.C:
        for j in model.C:
            if i != j:
                model.mtz.add(model.u[i, k] - model.u[j, k] + len(N) * model.x[i, j, k] <= len(N) - 1)

# Capacity constraints
model.Carga = ConstraintList()
for k in model.V:
    expr = sum(model.x[i, j, k] * model.De[j] for i in model.N for j in model.N if i != j)
    model.Carga.add(expr <= model.P[k])


# Distance constraints
model.Distancia = ConstraintList()
for k in model.V:
    expr = sum(model.x[i, j, k] * model.d[i, j, tipo_vehiculo[k-1] ] for i in model.N for j in model.N if i != j) 
    model.Distancia.add(expr <= model.M[k])



In [324]:
from amplpy import modules
import pyomo.environ as pyo
# solver_name = "highs"  # "highs", "cbc",  "couenne", "bonmin", "ipopt", "scip", or "gcg".
# solver = pyo.SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")


# solver.solve(model, tee=True,options = {
#     "time_limit": 10,
#     "presolve": "on",               # on/off
#     "mip_rel_gap": 0.01,            # 1% optimality gap
#     "mip_abs_gap": 1.0,             # absolute optimality gap
#     "random_seed": 42,              # reproducibility
#     "parallel": "on",               # parallel processing (default: on)
#     "threads": 8,                   # Usa múltiples hilos (si está disponible)

# })

# model.display()

solver = SolverFactory('gurobi')
results = solver.solve(
    model,
    tee=True,
    options={
        'TimeLimit': 30,
        'MIPGap': 0.01,
        'Threads': 4,
        'Presolve': 2,
        'Cuts': 2,
        'Heuristics': 0.5
    }
)

model.display()
print("Status:", results.solver.termination_condition)
print("Objective value:", model.obj())
print("Vehicle usage:")
for k in model.V:
    if model.ve[k].value > 0:
        print(f"Vehicle {k} is used.")
    else:
        print(f"Vehicle {k} is not used.")


print("Routes:")
for k in model.V:
    route = []
    for i in model.N:
        for j in model.N:
            if model.x[i, j, k].value is not None and model.x[i, j, k].value > 0:
                route.append((i, j))
    if route:
        print(f"Vehicle {k} route: {route}")

Read LP format model from file C:\Users\diego\AppData\Local\Temp\tmpvi3cji9h.pyomo.lp
Reading time = 0.02 seconds
x1: 357 rows, 400 columns, 2536 nonzeros
Set parameter TimeLimit to value 30
Set parameter MIPGap to value 0.01
Set parameter Threads to value 4
Set parameter Presolve to value 2
Set parameter Cuts to value 2
Set parameter Heuristics to value 0.5
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  30
MIPGap  0.01
Heuristics  0.5
Cuts  2
Presolve  2
Threads  4

Optimize a model with 357 rows, 400 columns and 2536 nonzeros
Model fingerprint: 0xfad3b615
Variable types: 0 continuous, 400 integer (364 binary)
Coefficient statistics:
  Matrix range     [9e-01, 3e+01]
  Objective range  [5e-01, 3e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range 